In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *

data = np.empty(shape=[0, 222])

all_data = glob.glob('./drive/MyDrive/compsci/test_unsampled_all_220.csv')

for j in all_data:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    data = np.append(data, csvrows, axis=0)

print(data.shape)

Loading  ./drive/MyDrive/compsci/test_unsampled_all_220.csv
(37863, 222)


In [ ]:
X = data[:,:-2]
y = data[:,-2]

In [ ]:
gbc_clf = GradientBoostingClassifier(n_estimators=100,random_state=48)
ada_clf = AdaBoostClassifier(n_estimators=100,random_state=48)
rfc_clf = RandomForestClassifier(max_depth=10, random_state=48,n_estimators=10)
NB_clf = GaussianNB()
NNMLP_clf = MLPClassifier(random_state=48, max_iter=100)
svc_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

n_folds = StratifiedKFold(n_splits = 6,shuffle= False ,random_state =48)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
f1_ada = cross_val_score(ada_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_ada score: ',f1_ada)
print('f1_ada score mean: ', f1_ada.mean())

precision_ada = cross_val_score(ada_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_ada score: ',precision_ada)
print('precision_ada score mean: ', precision_ada.mean())

recall_ada = cross_val_score(ada_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_ada score: ',recall_ada)
print('recall_ada score mean: ', recall_ada.mean())
print('--------------------------------------------------------------------------------------------------------')
f1_rfc = cross_val_score(rfc_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_rfc score: ',f1_rfc)
print('f1_rfc score mean: ', f1_rfc.mean())

precision_rfc = cross_val_score(rfc_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_rfc score: ',precision_rfc)
print('precision_rfc score mean: ', precision_rfc.mean())

recall_rfc = cross_val_score(rfc_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_rfc score: ',recall_rfc)
print('recall_rfc score mean: ', recall_rfc.mean())
print('--------------------------------------------------------------------------------------------------------')
f1_NB = cross_val_score(NB_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_NB score: ',f1_NB)
print('f1_NB score mean: ', f1_NB.mean())

precision_NB = cross_val_score(NB_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_NB score: ',precision_NB)
print('precision_NB score mean: ', precision_NB.mean())

recall_NB = cross_val_score(NB_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_NB score: ',recall_NB)
print('recall_NB score mean: ', recall_NB.mean())
print('--------------------------------------------------------------------------------------------------------')
f1_NNMLP = cross_val_score(NNMLP_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_NNMLP score: ',f1_NNMLP)
print('f1_NNMLP score mean: ', f1_NNMLP.mean())

precision_NNMLP = cross_val_score(NNMLP_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_NNMLP score: ',precision_NNMLP)
print('precision_NNMLP score mean: ', precision_NNMLP.mean())

recall_NNMLP = cross_val_score(NNMLP_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_NNMLP score: ',recall_NNMLP)
print('recall_NNMLP score mean: ', recall_NNMLP.mean())
print('--------------------------------------------------------------------------------------------------------')
f1_svc = cross_val_score(svc_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_svc score: ',f1_svc)
print('f1_svc score mean: ', f1_svc.mean())

precision_svc = cross_val_score(svc_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_svc score: ',precision_svc)
print('precision_svc score mean: ', precision_svc.mean())

recall_svc = cross_val_score(svc_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_svc score: ',recall_svc)
print('recall_svc score mean: ', recall_svc.mean())
print('--------------------------------------------------------------------------------------------------------')

f1_ada score:  [0.74997047 0.7488674  0.73808933 0.74343173 0.73513805 0.73526868]
f1_ada score mean:  0.7417942758047875


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

precision_ada score:  [0.70774413 0.69429255 0.72815035 0.69326666 0.68482055 0.70222464]
precision_ada score mean:  0.701749814279648
recall_ada score:  [0.8184123  0.81666931 0.80050705 0.81442155 0.79952456 0.79603803]
recall_ada score mean:  0.8075954678115497
--------------------------------------------------------------------------------------------------------
f1_rfc score:  [0.95979456 0.95982997 0.95616898 0.95656458 0.95845415 0.95797512]
f1_rfc score mean:  0.9581312271629915
precision_rfc score:  [0.96413712 0.96439033 0.96274174 0.96108432 0.96266871 0.96323917]
precision_rfc score mean:  0.963043564180713
recall_rfc score:  [0.96450642 0.96450642 0.96197116 0.96212361 0.96354992 0.96354992]
recall_rfc score mean:  0.963367908505119
--------------------------------------------------------------------------------------------------------
f1_NB score:  [0.53861543 0.52793149 0.51676814 0.52204895 0.53374185 0.52721936]
f1_NB score mean:  0.5277208678930995
precision_NB score:

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


f1_NNMLP score:  [0.97874442 0.98085548 0.98294407 0.98270644 0.98281387 0.97922047]
f1_NNMLP score mean:  0.9812141243033491


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


precision_NNMLP score:  [0.97931157 0.98101623 0.98286242 0.98259689 0.98297374 0.97907518]
precision_NNMLP score mean:  0.9813060056595729


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


recall_NNMLP score:  [0.97892569 0.98082713 0.98320393 0.98304279 0.98304279 0.9800317 ]
recall_NNMLP score mean:  0.9815123360606921
--------------------------------------------------------------------------------------------------------
f1_svc score:  [0.97819578 0.98073036 0.97549828 0.97644813 0.97903607 0.97878379]
f1_svc score mean:  0.9781154028260817
precision_svc score:  [0.97952525 0.98172519 0.9773267  0.97736188 0.98026152 0.97976255]
precision_svc score mean:  0.9793271818096799
recall_svc score:  [0.97971795 0.9819363  0.9774996  0.97797147 0.98034865 0.98019017]
recall_svc score mean:  0.9796106932419878
--------------------------------------------------------------------------------------------------------


In [ ]:
f1_gbc = cross_val_score(gbc_clf ,X,y,cv=n_folds,scoring ='f1_weighted')
print('f1_gbc score: ',f1_gbc)
print('f1_gbc score mean: ', f1_gbc.mean())

precision_gbc = cross_val_score(gbc_clf ,X,y,cv=n_folds,scoring ='precision_weighted')
print('precision_gbc score: ',precision_gbc)
print('precision_gbc score mean: ', precision_gbc.mean())

recall_gbc = cross_val_score(gbc_clf ,X,y,cv=n_folds,scoring ='recall_weighted')
print('recall_gbc score: ',recall_gbc)
print('recall_gbc score mean: ', recall_gbc.mean())